In [18]:
#Find a way to save the tweet to disk

import json
import oauth2 as oauth
import os
import emoji as emo
import urllib.parse as parse
import csv
from sklearn.feature_extraction.text import CountVectorizer
from stemming.porter2 import stem
from string import punctuation
import numpy
import pymongo
import re
import unicodedata
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

In [19]:
#This function will use an online ready-made dictionary to find the sentiment of an emoji

#load csv info into dictionary
emoji_scores = {}
with open('Emoji_Sentiment_Data_v1.0.csv', newline = '') as emoji_csv:
    emoji_reader = csv.reader(emoji_csv)
    next(emoji_reader, None)
    for row in emoji_reader:
        emoji = row[0]
        neg_score = float(int(row[4])/int(row[2]))
        neut_score = float(int(row[5])/int(row[2]))
        pos_score = float(int(row[6])/int(row[2]))
        sent_score = (-1 * neg_score + pos_score) - .305
        emoji_scores[emoji] = sent_score
        
#print(emoji_scores)
        

In [20]:
consumer_key = os.environ.get('CONSUMER_KEY')
consumer_secret = os.environ.get('CONSUMER_SECRET')

access_token = os.environ.get('ACCESS_TOKEN')
access_token_secret = os.environ.get('ACCESS_TOKEN_SECRET')

consumer = oauth.Consumer(key=consumer_key, secret=consumer_secret)
access_token = oauth.Token(key=access_token, secret=access_token_secret)
client = oauth.Client(consumer, access_token)

In [21]:
# text preprocessing (partially from before partially from August)
mongo_uri = 'mongodb://heroku_xgnhblcr:' + os.environ.get('MONGODB_PASSWORD') + '@ds149511.mlab.com:49511/heroku_xgnhblcr'
client = pymongo.MongoClient(mongo_uri)
db = client.get_default_database()
myresults = list(db.Justin_Bieber.find())
tweets = []
emoji_tweets = set()
for entry in myresults:
    tweets.append(entry['text'])
#Doing the next part based on the earlier REST-API-using cell
for tweet in tweets:
    tweet_emojis = []
    for char in tweet:
        if char in emo.UNICODE_EMOJI:
            tweet_emojis.append(char)
    if tweet_emojis:
        for emoji in tweet_emojis:
            tweet_emoji_scores = []
            try:
                tweet_emoji_scores.append(emoji_scores[emoji]) # emoji_scores from emoji dictionary
            except KeyError as e:
                continue
            if tweet_emoji_scores:
                if max(tweet_emoji_scores) == max(numpy.absolute(tweet_emoji_scores)):
                    tweet_score = max(tweet_emoji_scores)
                else:
                    tweet_score = min(tweet_emoji_scores)
                emoji_tweets_element = tweet, tweet_score
                emoji_tweets.add(emoji_tweets_element)
    
print(emoji_tweets)

{('Tweet Longer a Post: Squidward Is Now Justin Bieber|i Love Life (↺RT❤) https://t.co/JusNq5wYJs', 0.4410869565217392), ('great! 😇 😸 https://t.co/EosVToYWhO', 0.295), ('❤❤💙#JustinBieber #Music JUSTIN BIEBER Forever Junior Size XL White T-Shirt #Bieber #Memorabilia…… https://t.co/pm3578BmN5', 0.4410869565217392), ('Justin Bieber broke the world🌍 record and Twitter📳 record on Twitter😊 by having 💯million followers✔ 👏👏👏👏 he a ♛ all he needs is a🔱 #RESPECT', -0.22166666666666668), ('awesome.. 👍 😏 https://t.co/e59NfvAVSS', 0.21711434735706575), ('Zayn 😍 https://t.co/7VBYMIyNPx', 0.37293678251297374), ('@KennyBilski ❤💯 my idol is Justin Bieber .', -0.1841208791208791), ('I hope that one day i could meet BTS , Justin Bieber and Miley Cyrus. They are the ones who have inspired me 😊❤', 0.4410869565217392), ('Justin Bieber is my first love omg 😍❤\n#MPN #JustinBieberShow', 0.4410869565217392), ("I make your dreams come true when you wake up🎵-  I'm the One ft. Justin Bieber", 0.20172645739910317),

In [22]:
# Remove retweets (starting with "RT")
# emoji_tweets ust be a list of arbitrary order to be fed into the classifier
# note that emoji_tweets was originally a set
emoji_tweets = [tweet for tweet in emoji_tweets if not tweet[0].startswith('RT')]

tweet_text = [tweet[0] for tweet in emoji_tweets]
    

In [23]:
print(len(emoji_tweets)) #This is the tweets that are not duplicates and not retweets

2276


In [35]:
def preprocessor(tweet):
    
    #remove accents
    normalized = unicodedata.normalize('NFKD', tweet)
    if normalized != tweet:
        tweet = ''.join([c for c in normalized if not unicodedata.combining(c)])
        
    tweet = tweet.lower()
    
    #remove URLs
    tweet = re.sub(r'https?:\/\/.*?(\s|$)', '', tweet)
    tweet = re.sub(r'@\w*?(\s|$)', '', tweet)
    tweet = re.sub(r'#\w*?(\s|$)', '', tweet)
    
    #Note - the preprocessor already seems to be removing emojis - no need for extra code
    
    # Regularize apostrophes from different parts of unicode
    tweet = re.sub('|'.join(['‘', '’', 'ʻ', 'ʼ']), '\'', tweet)
    
    return tweet
    

In [36]:
def tokenizer(tweet): #Currently troubleshooting function
    
    tokens = []
    token_pattern = '(^|(?!\w).)((\w|\')+)\W'
    token_matches = re.finditer(token_pattern, tweet)
    for token_match in token_matches:
        tokens.append(token_match.group(2))
    return tokens

print(tokenizer("Hard pass...I'll give as much respect as y'all did Barack...none 😒 https://t.co/UJcr4mrZRA"))

['Hard', "I'll", 'as', 'respect', "y'all", 'Barack', 'none', 'https', 't']


In [37]:
vectorizer = CountVectorizer(preprocessor=preprocessor, tokenizer=tokenizer) #Make binary?  Does this prevent it from picking up one word twice in a tweet?
fit_tr = vectorizer.fit_transform(tweet_text)
print(vectorizer.get_feature_names())
#   for word in emoji_tweets.split(): #This and following line should be removed and use regex instead.  Follow Downloads/pak-paroubek.pdf for preprocessing ideas, as well as downloads that were downloaded at a similar time
#      if word[0:4] == 'http':
            
#CountVectorizer().build_tokenizer()(tweet_text) #stop_words param to 'english' once you have only Eng tweets, this was indented within for loop at the beginning of August

["'", "''malibu''", "'07", "'my", "'s", "'swap", "'violated'", "'when", '0', '00', '02', '03', '06', '07', '089', '1', '10', '100', '100m', '11', '12', '13', '15', '16', '17', '170912', '174', '18', '19', '1994', '1st', '2', '20', '200', '2010', '2012', '2015', '2016', '2017', '2017ss', '21', '23', '24', '25k', '28', '291', '29th', '2u', '3', '30', '31', '33', '34', '3d', '4', '44', '450', '5', '50', '5sos', '6', '600k', '7', '76k', '8', '84839292', 'a', 'a03', 'aaaaa', 'aarif', 'abbie', 'about', 'abs', 'absolutely', 'accounts', 'acoustic', 'actor', 'actually', 'adam', 'added', 'addict', 'adesso', 'adorable', 'after', 'again', 'agree', 'ah', 'ahahah', 'ahhh', 'ahhhhhhhh', 'aire', 'alan', 'alasdair', 'album', 'alexander', 'alike', 'alive', 'all', 'allowed', 'already', 'also', 'always', 'am', 'amazing', 'amd', 'amount', 'amp', 'an', 'and', 'angel', 'angels', 'announcement', 'another', 'any', 'anymore', 'anyone', 'anything', 'app', 'apparently', 'apple', 'appointment', 'appropriating', 'a

In [38]:
transformer = TfidfTransformer()  # This block is not correct
tfidf = transformer.fit_transform(fit_tr)

In [39]:
#Tweets must have arbitrary order to be used with Multinomial NB


tweet_emos = []
for tweet in emoji_tweets:
    tweet_emos.append(tweet[1] > 0)
        
    
    #Cross-validation
from sklearn.model_selection import cross_val_score
print(type(tfidf.toarray()))
          
scores = cross_val_score(MultinomialNB(alpha=.01), tfidf, tweet_emos, cv=5) #this takes forever at cv=10
print(scores)  
    
    

<class 'numpy.ndarray'>
[ 0.71929825  0.76703297  0.76703297  0.76263736  0.76923077]


In [17]:
    
    
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    MultinomialNB(alpha=.01).fit(X_train, y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

________________________________________________________________________________
Training: 


NameError: name 'clf' is not defined